In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Debug\net5.0\bossspad.dll"

//#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#18.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Open Database

In [ ]:
BoSSSshell.WorkflowMgm.Init("CoFloFlame_BS_22211");

In [ ]:
 static var myBatch = GetDefaultQueue();
 (myBatch as MsHPC2012Client).ComputeNodes = new string[]{"HPCLUSTER3"};

static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

## Case configuration

In [ ]:
int NumberOfProcMF = 8;
int dgMF = 2; // Mass fraction DG degree
int nCells = 20;


bool BurkeSchu = true; // Configuration for the BS flame => constant density, uniform velocity inlet, 

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid(int nCells,double r, double R) {
        // Geometry
        // All lenghts are non.dimensionalized with Lref = fuel inlet radius (0.2 cm)
        double LRef = R;

        
        double xmin = -R / LRef;
        double xmax = +R / LRef;
        double ymin = 0;
        double zlength = R*5;
        double ymax = zlength / LRef;
        double rAd = r / LRef;

        double leftmidpoint = (xmin - rAd) / 2;
        double rightmidpoint = (xmax + rAd) / 2;

        var troubledPoints = new double[][] {
        new double[]{ rAd, 0.0 },
        new double[]{-rAd, 0.0 }
        };
        double stretchfactorY = 0.98 * 1;

        double sf1 = 0.97 * 1 * 0;
        double sf2 = 0.95 * 1 * 0;
        double sf3 = 0.97 * 1 * 0;
        int n1 = (int)2.5 * nCells;
        int n2 = (int)1.0 * nCells;
        int n3 = (int)2.5 * nCells;
        //var xNodes1 = GenericBlas.SinLinSpacing(xmin, -rAd, sf1, n1 + 1);
        //var xNodes2 = GenericBlas.SinLinSpacing(-rAd, rAd, sf2, n2 + 1);
        //var xNodes3 = GenericBlas.SinLinSpacing(rAd, xmax, sf3, n3 + 1);

        List<double> xNodes2 = (GenericBlas.SinLinSpacing(-rAd, rAd, sf2, n2 + 1)).ToList(); // nodes corresponding to the fuel inlet
        List<double> xNodes3 = (GenericBlas.SinLinSpacing(rAd, (xmax - rAd) * 2 + rAd, sf3, n1 * 2 + 1).ToList()); // Nodes corresponding to the oxidizer inlet, right part
        var myXnodes3 = xNodes3.GetSubVector(0, xNodes3.Count / 2 + 1); // Take only "left side" of node array
        var myxNodes1 = myXnodes3.CloneAs();
        myxNodes1.ScaleV(-1.0);
        Array.Reverse(myxNodes1);

        List<double> list2 = new List<double>();
        list2.AddRange(myxNodes1.Take(n1 + 0).ToList());
        list2.AddRange(xNodes2.Take(n2 + 0).ToList());
        list2.AddRange(myXnodes3.Take(n3 + 1).ToList());
        double[] _xNodes = list2.ToArray();
        //Debug.Assert(_xNodes.Contains(xmin));
        //Debug.Assert(_xNodes.Contains(-rAd));
        //Debug.Assert(_xNodes.Contains(rAd));
        //Debug.Assert(_xNodes.Contains(xmax));
        var _yNodes = GenericBlas.SinLinSpacing(ymin, ymax * 2, stretchfactorY, (2 * nCells) * 6 + 1);
        var myYnodes = _yNodes.GetSubVector(0, _yNodes.Length / 2 + 1); // I just want a fine mesh in the bottom part of the grid.
                                                                        //var _xNodes = GenericBlas.Linspace(xmin, xmax, 2 * meshScaling + 1);
                                                                        //var myYnodes = GenericBlas.Linspace(ymin, ymax, 2 * meshScaling + 1);
        var grd = Grid2D.Cartesian2DGrid(_xNodes, myYnodes, periodicX: false);
        grd.EdgeTagNames.Add(1, "Velocity_Inlet_O2");
        grd.EdgeTagNames.Add(2, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");
        grd.EdgeTagNames.Add(4, "wall_moving");
        //grd.EdgeTagNames.Add(4, "NoSlipNeumann");
        // grd.EdgeTagNames.Add(4, "Velocity_Inlet_outer"); // We want a constant velocity field.

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //Edge tags
            //1: Velocity inlet O_2
            //2: Velocity inlet CH_4
            //3: Pressure outlet

            //Inlet fuel
            if(Math.Abs(y - ymin) < 1e-8 && Math.Abs(x - 0.0) < rAd + 1e-8)
                return 2;
            //Inlet oxidizer
            if((Math.Abs(y - ymin) < 1e-8 && ((Math.Abs(x - rightmidpoint) < ((xmax - rAd) * 0.5 + rAd) + 1e-8) || Math.Abs(x - leftmidpoint) < Math.Abs((xmin - rAd) * 0.5 + rAd) + 1e-8)))
                return 1;

            // Pressure outlet

            if((Math.Abs(y - ymax) < 1e-8))
                return 3;

            // Pressure outlet

            if((Math.Abs(x - xmin) < 1e-8) || (Math.Abs(x - xmax) < 1e-8))
                return 4;
            else throw new ArgumentOutOfRangeException();
        
        }
        );
        myDb.SaveGrid(ref grd);

        return grd;
    }
}

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma) {
        using(var stw = new System.IO.StringWriter()) {

            stw.WriteLine("static class BoundaryValues {");

            stw.WriteLine("  static public double ConstantValue(double[] X) {");
            stw.WriteLine("    return "+ ConstVal +";");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuel(double[] X) {");
            stw.WriteLine("    return  (1.0 - Math.Pow(X[0] / "+inletRadius+", 2)) * "+uInFuel+" ;");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuelOscilatory(double[] X, double t) {");
            stw.WriteLine("    return  (1.0 + 3.7*Math.Sin(3.14*t) )*(1.0 - Math.Pow(X[0] / "+inletRadius+", 2)  ) * "+uInFuel+" ;");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuelOscilatory_tref(double[] X, double t) {");
            stw.WriteLine(" double t2 = t/"+sigma+";"); // sigma is the reference time, Lref/uref
            stw.WriteLine("    return  (1.0 + 0.7*Math.Sin(3.14*t2) )*(1.0 - Math.Pow(X[0] / "+inletRadius+", 2)  ) * "+uInFuel+" ;");
            stw.WriteLine("  }");





            stw.WriteLine("}"); 
            return stw.ToString();
        }
    }

    static public Formula Get_ConstantValue(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ConstantValue", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_ParabolaVelocityFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_ParabolaVelocityFuelOscilatory(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuelOscilatory",TimeDep:true, AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }
    static public Formula Get_ParabolaVelocityFuelOscilatory_tref(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuelOscilatory_tref",TimeDep:true, AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }
}

## Create base control file
In this ControlFile basic configuration of the CounterDiffusionFlame is defined.

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int nCells, bool isMF) {
    var CC = new ChemicalConstants();
    
    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();

    C.NumberOfChemicalSpecies = 4; 
    C.SetDGdegree(dg); //


    
    double h =0.1/100 ; // Radius inner , m
    // double R =(1.0)*2.54/100*4 ; // Radius outter cylinder, m
    double r = h;
    
    double R = r/0.01 ; // Radius outter cylinder, m
    

    C.SetGrid(GridFactory.GenerateGrid(nCells,r,R));  //
    C.MatParamsMode = MaterialParamsMode.Constant;      
    C.rhoOne = true;
    // Problem Definition
    //===================
    double TemperatureInFuel = 300;  //
    double TemperatureInOxidizer = 300; //
    double AtmPressure = 101325; // Pa
    // double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };
    double[] FuelInletConcentrations = new double[] {0.2, 0.0, 0.0, 0.0, 0.8 };

    double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; //
    double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };

    double mwFuel = CC.getAvgMW(MWs, FuelInletConcentrations);
    double mwAir = CC.getAvgMW(MWs, OxidizerInletConcentrations);
    double densityAirIn = AtmPressure * mwAir / (CC.R_gas * TemperatureInOxidizer * 1000); // kg / m3
    double densityFuelIn = AtmPressure * mwFuel / (CC.R_gas * TemperatureInFuel * 1000); // kg / m3.

    double uInFuel = 1.33*2.54/100;
    double uInAir = uInFuel;

    Console.WriteLine("VelocityFuel" + uInFuel);
    Console.WriteLine("VelocityAir" + uInAir);

    // Reference values
    //===================
    // Basic units to be used: Kg, m, s, mol, pa,
    double TRef = TemperatureInOxidizer;// Reference temperature  is the inlet temperature, (K)
    double pRef = AtmPressure; // Pa
    double uRef = Math.Max(uInFuel, uInAir); // m/s

    double LRef = R;

    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 }; //No gravity.

    // Solver configuration
    // =======================
    C.smoothingFactor = 0;
    
    
    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.MaxSolverIterations = 10;
    
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.saveperiod = 1;
    C.PenaltyViscMomentum = 1.0; 
    C.PenaltyHeatConduction = 1.0;
    C.YFuelInlet = FuelInletConcentrations[0];
    C.YOxInlet = OxidizerInletConcentrations[1];
    C.FuelInletConcentrations = FuelInletConcentrations;
    C.OxidizerInletConcentrations = OxidizerInletConcentrations;
    C.TFuelInlet = 1.0;
    C.TOxInlet = 1.0;
    C.PhysicalParameters.IncludeConvection = true;
    // Chemical related parameters
    double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
    C.phi = s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
 

    //==========================
    //Derived reference values
    //==========================
    C.uRef = uRef; // Reference velocity
    C.LRef = LRef; // reference length
    C.pRef = AtmPressure; // reference pressure
    C.TRef = TemperatureInFuel;// reference temperature
    C.MWRef = 1.0;//
    C.rhoRef =1.0 ;
    C.cpRef = 1.0;
    C.muRef = 4.922e-5; //m^2/s
    C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
    C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses
    C.T_ref_Sutherland = 300;

    double heatRelease_Ref = (C.TRef * C.cpRef);
    C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
    C.B = CC.PreExponentialFactor;

    C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };

    C.Reynolds =  100;//C.uRef * C.LRef / C.muRef;
    C.Prandtl = 1.0; 
    C.Schmidt = C.Prandtl; // Because Lewis number  is assumed as 1.0  (Le = Pr/Sc)

    C.Lewis = new double[] { 1.0, 1.0, 1.0, 1.0, 1.0 };

    double g = 9.8; // m/s2
    C.Froude = Math.Sqrt(uRef * uRef / (C.LRef * g)); // Not used
    C.T_ref_Sutherland = 300; 
    C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 }; 




    //==========================
    // Initial conditions
    //==========================
    double dummy = 0;
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, uInFuel / C.uRef, uInAir / C.uRef,dummy));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, uInFuel / C.uRef, uInAir / C.uRef,dummy));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, uInFuel / C.uRef, uInAir / C.uRef,dummy));

    //==========================
    // Boundary conditions
    //==========================

 

    
    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    // C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ParabolaVelocityFuel( dummy, r/C.LRef, uInFuel / uRef, uInAir / uRef, dummy));

    
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy,dummy, dummy));

    C.AddBoundaryValue("wall_moving", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    C.AddBoundaryValue("wall_moving", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy,dummy, dummy));

  
    // C.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy,dummy, dummy));
    // C.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue( uInAir / uRef, dummy, dummy,dummy, dummy));

    Console.WriteLine("Re:" + C.Reynolds);
    return C;
}

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true  );

    
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "CoFlowFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells+"daa1b1211a17e";
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.UseSelfMadeTemporalOperator = false;
    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 30; 
    
    // Boundary and initial conditions
    double dummy = -11111111;

    C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));
    
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));        
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(0.0,dummy,dummy , dummy, dummy));

    // C_MixtureFraction.AddBoundaryValue("wall_moving", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(0.0,dummy,dummy , dummy, dummy));


    
   
    int NoOfPseudoTimesteps =  3;
    C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 
    C_MixtureFraction.NoOfTimesteps = NoOfPseudoTimesteps ;


    C_MixtureFraction.AdaptiveMeshRefinement = true; 
    C_MixtureFraction.AMR_startUpSweeps = 3;



    // C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onProblematicPoints(troubledPoints,C_MixtureFraction.AMR_startUpSweeps) );  
    // C_MixtureFraction.activeAMRlevelIndicators.Add(new BoSSS.Application.XNSEC.AMR_RefineAroundProblematicPoints(troubledPoints, 3, 0.2));
    C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_MixtureFraction.zSt,3) );
    return C_MixtureFraction;
}

In [ ]:
myDb.Sessions[0].Export().Do()

C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\CoFloFlame_BS_22211__FS_CoFlowFlameP2K20daa1b1211a17e__6b299bf7-bc72-44de-8467-9ebf4daada0d

## Send and run jobs

In [ ]:
Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
bool trefok = false; // shouldnt matter
var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dgMF, nCells); 
string jobName       = C_MixtureFraction.SessionName;
Console.WriteLine(jobName);
var oneJob           = new Job(jobName, solver_MF);
oneJob.NumberOfMPIProcs = NumberOfProcMF;
oneJob.NumberOfThreads = 1;
oneJob.SetControlObject(C_MixtureFraction);
oneJob.Activate(myBatch); 


In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();


In [ ]:
myDb.Sessions[0].Export().Do()

In [ ]:
myDb

In [ ]:
C_MixtureFraction.uRef

In [ ]:
C_MixtureFraction.LRef

In [ ]:
C_MixtureFraction.rhoRef

In [ ]:
C_MixtureFraction.muRef